In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
import json
from dateutil.parser import *
import matplotlib.pyplot as plt
import time
from dateutil import tz
import pytz
import plotly.graph_objects as go
import plotly.express as px

In [2]:
#suwon2912
API_KEY = '071bd4342fc862e3e4903c64894ea1ce-30997bfa44030c591f512e4e0b7a35b1'
ACCOUNT_ID = '101-003-19563483-001'
OANDA_URL = 'https://api-fxpractice.oanda.com/v3'

SECURE_HEADER = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

In [3]:
session = requests.Session()

def get_his_data_filename(pair, granularity):
    return f"downloaded data/{pair}_{granularity}_break_plot.pkl"

def astimezone(row):
    return row.astimezone(pytz.timezone('Singapore'))

def parser(row):
    return parse(row)

In [4]:
data_library = {
    'GBP_USD' :  { "pair" :  'GBP_USD', "units": 6000, "pip": 0.0001, "pips_tp": 55, "pips_sl": 20, "pips_exceed": 5,"pips_bound": 10},
    'USD_JPY' :  { "pair" :  'USD_JPY', "units": 6000, "pip": 0.01, "pips_tp": 55, "pips_sl": 20, "pips_exceed": 5,"pips_bound": 10},
    'AUD_JPY' :  { "pair" :  'AUD_JPY', "units": 3000, "pip": 0.01, "pips_tp": 110, "pips_sl": 40, "pips_exceed": 10,"pips_bound": 15},
    'USD_CAD' :  { "pair" :  'USD_CAD', "units": 3000, "pip": 0.0001, "pips_tp": 110, "pips_sl": 40, "pips_exceed": 10,"pips_bound": 15},
    'GBP_JPY' :  { "pair" :  'GBP_JPY', "units": 3000, "pip": 0.01, "pips_tp": 110, "pips_sl": 40, "pips_exceed": 10,"pips_bound": 15},
    'XAU_USD' :  { "pair" :  'XAU_USD', "units": 3,    "pip": 0.1, "pips_tp": 110, "pips_sl": 40, "pips_exceed": 10,"pips_bound": 15},
    'BCO_USD' :  { "pair" :  'BCO_USD', "units": 30,    "pip": 0.01, "pips_tp": 110, "pips_sl": 40, "pips_exceed": 10,"pips_bound": 15},
    'WTICO_USD' :  { "pair" :  'WTICO_USD', "units": 30,    "pip": 0.01, "pips_tp": 110, "pips_sl": 40, "pips_exceed": 10,"pips_bound": 15}

    }

data_plot_library = dict(
    GBP_USD = dict(pips = 0.0001, pips_tp = 50, pips_sl = 25),
    USD_JPY = dict(pips = 0.01,pips_tp = 50, pips_sl = 25),
    AUD_JPY = dict(pips = 0.01,pips_tp = 100, pips_sl = 50),
    USD_CAD = dict(pips = 0.0001,pips_tp = 100, pips_sl = 50),
    GBP_JPY = dict(pips = 0.01,pips_tp = 100, pips_sl = 50),
    XAU_USD = dict(pips = 0.1,pips_tp = 100, pips_sl = 50),
    BCO_USD = dict(pips = 0.01,pips_tp = 100, pips_sl = 50),
    WTICO_USD = dict(pips = 0.01,pips_tp = 100, pips_sl = 50)
)

In [5]:
def fetch_candles(pair_name, count, granularity):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, headers=SECURE_HEADER)
    return response.status_code, response.json()

def get_candles_df(json_response):

    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    df = pd.DataFrame.from_dict(our_data)
    df["time"] = [parse(x) for x in df.time]
    return pd.DataFrame.from_dict(our_data)

def save_file(candles_df, pair, granularity):
    candles_df.to_pickle(get_his_data_filename(pair, granularity))
    
def create_data(pair, granularity = 'M30'):
    code, json_data = fetch_candles(pair, 2000, granularity)
    if code != 200:
        print(pair, "Error")
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

pair = 'USD_CAD'
create_data(pair)

#['AUD_JPY', 'GBP_USD', 'GBP_JPY', 'USD_JPY', 'USD_CAD', 'XAU_USD', 'BCO_USD', 'WTICO_USD']

USD_CAD loaded 1999 candles from 2021-09-13T09:30:00.000000000Z to 2021-11-10T01:30:00.000000000Z


In [6]:
def plot_stop_log(pair,granularity = 'M30', time_delta = 46):
    
    df = pd.read_pickle(get_his_data_filename(pair, granularity))
    non_cols = ['time','volume']
    mod_cols = [x for x in df.columns if x not in non_cols]
    df[mod_cols]=df[mod_cols].apply(pd.to_numeric)
    df.reset_index(inplace=True,drop=True)

    df['time'] = df['time'].apply(parser)

    df['time'] = df['time'].apply(astimezone)
    
    df_plot = df.iloc[-1600:]
    df_plot.reset_index(inplace=True,drop=True)
    
    fig = go.Figure()
    fig.add_trace(go.Candlestick(x = df_plot.time,
                                 open = df_plot.mid_o,
                                 high = df_plot.mid_h,
                                 low = df_plot.mid_l,
                                 close=df_plot.mid_c))
    
    fig.update_layout(width=950,height=500,
                 margin = dict(l=10,r=10,t=10,b=10))
    
    log_df = pd.read_pickle('break_stop_log.pkl')
    log_df.reset_index(inplace=True,drop=True)
    
    for i in log_df.index:
        if log_df['PAIR'][i] == pair:
            if log_df['time'][i]> (dt.datetime.now()- dt.timedelta(time_delta)).astimezone(pytz.timezone('Singapore')):
                if log_df['decision'][i] ==1:
                    fig.add_shape(type="line",
                        x0=log_df['time'][i], 
                        y0=log_df['sure_level'][i], 
                        x1=df_plot['time'].iloc[-1], 
                        y1= log_df['sure_level'][i],
                        line=dict(color="red",width=1))

    for i in log_df.index:
        if log_df['PAIR'][i] == pair:
            if log_df['time'][i]> (dt.datetime.now()- dt.timedelta(time_delta)).astimezone(pytz.timezone('Singapore')):
                if log_df['decision'][i] ==-1:
                    fig.add_shape(type="line",
                    x0=log_df['time'][i], 
                    y0=log_df['sure_level'][i], 
                    x1=df_plot['time'].iloc[-1], 
                    y1=log_df['sure_level'][i],
                    line=dict(color="RoyalBlue",width=1))

    fig.update_xaxes(
            gridcolor="#1f292f",
            showgrid=False,fixedrange=True,rangeslider=dict(visible=True)
            #,rangebreaks=[
                #dict(bounds=["sat", "mon"])]
                )
    
    # for trade history
    df_trade = pd.read_csv('/Users/User2/Desktop/Pepper Statement.csv')
    df_trade_pair = df_trade[df_trade['Symbol']==pair].copy()
    
    df_trade_pair = df_trade_pair[['Symbol','Opening Direction', 'Opening Time', 'Closing Time', 'Entry Price','Closing Price', 'Closing Quantity','Net SGD']].copy()
    df_trade_pair['Opening Time'] = df_trade_pair['Opening Time'].apply(parser)
    df_trade_pair['Closing Time'] = df_trade_pair['Closing Time'].apply(parser)
    df_trade_pair = df_trade_pair[df_trade_pair['Opening Time']> dt.datetime.now()- dt.timedelta(time_delta)].copy()
    
    pair_take_profit = data_plot_library[pair]['pips'] * data_plot_library[pair]['pips_tp']
    pair_stop_loss = data_plot_library[pair]['pips'] * data_plot_library[pair]['pips_sl']

    df_trade_pair['Take_Profit'] = np.where(df_trade_pair['Opening Direction']=='Buy',
                                    df_trade_pair['Entry Price'] + pair_take_profit,
                                    df_trade_pair['Entry Price'] - pair_take_profit)

    df_trade_pair['Stop_Loss'] = np.where(df_trade_pair['Opening Direction']=='Buy',
                                    df_trade_pair['Entry Price'] - pair_stop_loss,
                                    df_trade_pair['Entry Price'] + pair_stop_loss)

    for i in df_trade_pair.index:
        if df_trade_pair['Opening Direction'][i] == 'Buy':
            if df_trade_pair['Opening Time'][i]> dt.datetime.now()- dt.timedelta(time_delta):
                fig.add_shape(type="line",
                x0=df_trade_pair['Opening Time'][i], 
                y0=df_trade_pair['Entry Price'][i], 
                x1=df_trade_pair['Closing Time'][i], 
                y1=df_trade_pair['Closing Price'][i],
                line=dict(color="Gold",width=2))         

    for i in df_trade_pair.index:
        if df_trade_pair['Opening Direction'][i] == 'Sell':
            if df_trade_pair['Opening Time'][i]> dt.datetime.now()- dt.timedelta(time_delta):
                fig.add_shape(type="line",
                x0=df_trade_pair['Opening Time'][i], 
                y0=df_trade_pair['Entry Price'][i], 
                x1=df_trade_pair['Closing Time'][i], 
                y1=df_trade_pair['Closing Price'][i],
                line=dict(color="Black",width=2))

    fig.update_layout(hovermode='x')
    fig.show()

    display(df_trade_pair)

plot_stop_log(pair)

,Symbol,Opening Direction,Opening Time,Closing Time,Entry Price,Closing Price,Closing Quantity,Net SGD,Take_Profit,Stop_Loss


In [7]:
df = pd.read_pickle('break_stop_log.pkl')

#df = df[df['PAIR']=='USD_JPY']

df.reset_index(inplace=True,drop=True)

df[df['PAIR']=='XAU_USD']


,time,PAIR,sure_level,decision,date_added
27,2021-05-28 20:11:00+08:00,XAU_USD,1883.650,-1,2021-05-28 20:11:00+08:00
28,2021-06-04 09:57:00+08:00,XAU_USD,1859.090,-1,2021-06-04 09:57:00+08:00
29,2021-06-10 20:31:00+08:00,XAU_USD,1871.030,-1,2021-06-10 20:31:00+08:00
30,2021-06-14 20:50:00+08:00,XAU_USD,1845.106,-1,2021-06-14 20:50:00+08:00
31,2021-06-17 23:31:00+08:00,XAU_USD,1768.070,-1,2021-06-17 23:31:00+08:00
...,...,...,...,...,...
341,2021-10-22 23:01:00+08:00,XAU_USD,1813.330,1,2021-10-26 10:32:33.104967
349,2021-10-26 22:25:00+08:00,XAU_USD,1782.990,-1,2021-10-28 19:07:09.935044
356,2021-10-28 21:31:00+08:00,XAU_USD,1809.445,1,2021-10-29 22:46:32.024690
360,2021-10-29 22:00:00+08:00,XAU_USD,1772.315,-1,2021-11-02 12:33:39.497467
